### Assignment 3

In [2]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline

Use tools in scipy.optimize (or elsewhere) to fit a linear model with a single variable 

 $$ AE = c_1 B_z $$

by finding the value of $c_1$ which minimizes the goodness of fit given by 

 $$ \chi^2 = \sum (\mathrm{model} - \mathrm{data})^2 $$

then try two variables

 $$ AE = c_1 B_z + c_2 v_x $$
 
then try a non-linear transformation of a single variable
 
 $$ AE = c_1 B_z + c_2 v_x^2 $$ 
 
then try cross terms 

 $$ AE = c_1 B_z + c_2 v_x + c_3 B_z v_x $$ 
 
then explore combinations and transformations of variables to obtain "the best" model.  Think about what that means.

In [3]:
data = np.load('/tmp/omni_1min_2014.npy')
plt.plot( data['bz'], data['ae_index'], '.', alpha=0.2 )

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/omni_1min_2014.npy'